In [ ]:
from Crypto.Util.number import *
from pwn import *
import json

r = remote('socket.cryptohack.org', 13376, level = 'DEBUG')

r.recv()
r.sendline(b'{\"option\": \"get_pubkey\"}')
data = json.loads(r.recv().decode())
n = int(data["N"],16)
e = int(data["e"],16)

target = b"admin=True"
target_num = bytes_to_long(target)

r.sendline(b'{\"option\": \"sign\", \"msg\": \"'+ hex(2*target_num)[2:].encode() + b'\"}')

data = json.loads(r.recv().decode())
s22 = int(data["signature"],16)

r.sendline(b'{\"option\": \"sign\", \"msg\": \"02\"}')
data = json.loads(r.recv().decode())
s2 = int(data["signature"],16)

s = hex((pow(s2,-1,n)*s22)%n)

r.sendline(b'{\"option\": \"verify\", \"msg\": \"' + hex(target_num)[2:].encode() + b'\", \"signature\": \"'+ s[2:].encode() + b'\"}')
print(r.recv())

## crypto{m4ll34b1l17y_c4n_b3_d4n63r0u5}
# This script implements the Franklin-Reiter related-message attack on a cryptographic challenge. It connects to a remote server to retrieve two encrypted messages with different padding values and the modulus n. Using these, it applies a polynomial-based GCD calculation to recover the plaintext message. The compositeModulusGCD function computes the GCD of two polynomial expressions, and the FranklinReiter function uses this to extract the original message, which is then printed in bytes form. The solution reveals the flag crypto{linear_padding_isnt_padding}.


[x] Opening connection to socket.cryptohack.org on port 13376
[x] Opening connection to socket.cryptohack.org on port 13376: Trying 134.122.111.232
[+] Opening connection to socket.cryptohack.org on port 13376: Done
[DEBUG] Received 0x21 bytes:
    b'Watch out for the Blinding Light\n'
[DEBUG] Sent 0x19 bytes:
    b'{"option": "get_pubkey"}\n'
[DEBUG] Received 0x21c bytes:
    b'{"N": "0x954e1412ba207b8a246ea515e81425aeb5471cf5062b6497b2c76312ccf150498779ca540464b09fe573df68b0cfdcac124ba799b8546b45b49eaae9fadd630d1b5562a9993c6a3da72d5222e24aa6e1f9c663bfd07f31f0cdef87a54f2fbf7151afc3fd329bd16692dcfa6794c3d94d00fb2e11b49557a491be3e510f0c3e22163487df65e54d68f43a3ecea44e48dc929f2d321c6bfdb2c6c233c704e0618041ace0be91f637f423e6161b36a1fe0f04445ee1f48dc5960659706bbcb97c1667c5f17d0f2395dad348a88f3efb7fa06f99f7963749679eb697cd178fce6f65cfee5b6c9c36096c96f5b5532a6a3b44127afe27f10015dd71a644d455f800d5", "e": "0x10001"}\n'
[DEBUG] Sent 0x32 bytes:
    b'{"option": "sign", "msg": "c2c8dad2dc7aa8e4e